<a href="https://colab.research.google.com/github/lauraluebbert/test_gget_alphafold/blob/main/test_gget_alphafold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [gget alphafold](https://github.com/pachterlab/gget) test

___

Install and import gget:

In [ ]:
!pip install -U -q gget
import gget

Install OpenMM:  

In [ ]:
# For pretty plots
%config InlineBackend.figure_format='retina'

# Install mamba through miniconda
import os
import sys
python_version = f"{sys.version_info.major}.{sys.version_info.minor}"
os.system("wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh")
os.system("bash Miniforge3-Linux-x86_64.sh -bfp /usr/local")
os.system("mamba config --set auto_update_conda false")
# Install openmm
os.system(f"mamba install -y -c conda-forge -c bioconda openmm=8.2.0 python='{python_version}'")
# Add to path
sys.path.append(f'/usr/local/lib/python{python_version}/site-packages')

Install third-part dependencies and download AlphaFold model parameters using `gget setup` (this might take a few minutes):

In [ ]:
gget.setup("alphafold")

Predict the 3D structure of CASP14 target [T1024](https://predictioncenter.org/casp14/target.cgi?id=8&view=all):

In [ ]:
gget.alphafold(
    "MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH",
    relax=True,
    out=None
    )